In [1]:
import os
import re
import unicodedata
import pandas as pd
import numpy as np
import torch
import gc
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM, 
    DataCollatorForSeq2Seq, 
    Seq2SeqTrainingArguments, 
    Seq2SeqTrainer
)
from datasets import Dataset, DatasetDict

# Clear memory first
gc.collect()
torch.cuda.empty_cache()

# Hardware check
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
if device == "cuda":
    print(f"GPU Count: {torch.cuda.device_count()}")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

2026-01-18 11:18:56.998087: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768735137.197757      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768735137.251906      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768735137.751146      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768735137.751183      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768735137.751187      55 computation_placer.cc:177] computation placer alr

Using device: cuda
GPU Count: 2
GPU Name: Tesla T4


In [2]:

TRAIN_FILE = "/kaggle/input/deep-past-initiative-machine-translation/train.csv"

MODEL_PATH = '/kaggle/input/akkadian-byt5-translator/pytorch/default/1/akkadian_byt5_trained'


MAX_LENGTH = 384       
BATCH_SIZE = 20         
GRAD_ACCUMULATION = 8  
LEARNING_RATE = 0.25e-3   
NUM_EPOCHS = 5
SEED = 42

torch.manual_seed(SEED)
np.random.seed(SEED)

In [4]:
def clean_text(text):
    if pd.isna(text):
        return ""
    
    text = str(text)
    
    
    text = unicodedata.normalize('NFC', text)
    
    
    text = text.replace("…", "...")
    text = re.sub(r'\.\s*\.\s*\.', '...', text) 
    
    
    text = text.replace('“', '"').replace('”', '"')
    
    
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def is_valid_translation(text):
    
    if not text or len(text) < 2:
        return False
    
    
    clean_check = re.sub(r'[^\w\s]', '', text)
    if len(clean_check) == 0:
        return False
        
    return True

def remove_noisy(df):
    oare_ids = []
    with open('/kaggle/input/outliers/oare_ids_outilers_iqr.txt','r') as f:
        oare_ids = [line.strip() for line in f.readlines()]
    mask = ~df['oare_id'].isin(oare_ids)
    return mask
    
def preprocess_dataframe(df):
    print(f"Original Row Count: {len(df)}")
    
    
    df['transliteration'] = df['transliteration'].apply(clean_text)
    df['translation'] = df['translation'].apply(clean_text)
    df_clean = df[remove_noisy(df)]
    
    valid_mask = df['translation'].apply(is_valid_translation)
    df_cleaned = df_clean[valid_mask].copy()
    
    print(f"Cleaned Row Count: {len(df_clean)}")
    print(f"Dropped {len(df) - len(df_clean)} rows (empty/broken translations)")
    
    return df_cleaned

In [5]:

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, local_files_only=True)


def preprocess_function(examples):
    # Prefix
    inputs = ["translate Akkadian to English: " + str(ex) for ex in examples["transliteration"]]
    targets = [str(ex) for ex in examples["translation"]]
    
    # Tokenize inputs
    model_inputs = tokenizer(
        inputs, 
        max_length=MAX_LENGTH, 
        truncation=True,
        padding=False # Dynamic padding is faster
    )
    
    # Tokenize targets
    labels = tokenizer(
        text_target=targets, 
        max_length=MAX_LENGTH, 
        truncation=True,
        padding=False
    )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Map


In [6]:
print(f"Loading model from {MODEL_PATH}...")


model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_PATH, 
    local_files_only=True,
    use_safetensors=True 
)
model = model.to(device)

# Collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Loading model from /kaggle/input/akkadian-byt5-translator/pytorch/default/1/akkadian_byt5_trained...


In [11]:
args = Seq2SeqTrainingArguments(
    output_dir="flan-T5-small-optimised",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    
    # MEMORY SAVING SETTINGS
    per_device_train_batch_size=BATCH_SIZE,        # 2
    per_device_eval_batch_size=BATCH_SIZE,         # 2
    gradient_accumulation_steps=GRAD_ACCUMULATION, # 8
    gradient_checkpointing=True,                   # <--- THE MAGIC FIX (Saves 50% VRAM)
    optim="adafactor",                             # <--- Uses less memory than AdamW
    lr_scheduler_type='cosine',
    weight_decay=0.02,
    save_total_limit=1,
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    metric_for_best_model="bleu",
    fp16=True,                  
    report_to="none",
    load_best_model_at_end=True,
    logging_steps=5,

    generation_max_length=MAX_LENGTH,        
    generation_num_beams=4,
)

import numpy as np
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)

    # 1. Decode predictions
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # 2. Decode labels (replace -100 padding with pad_token_id)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # 3. NLTK Formatting: Tokenize and wrap references in a list
    # BLEU expects: 
    #   References: [[['ref1_word1', 'ref1_word2']], [['ref2_word1', ...]]]
    #   Candidates: [['cand1_word1', 'cand1_word2'], ['cand2_word1', ...]]
    formatted_preds = [pred.strip().split() for pred in decoded_preds]
    formatted_labels = [[label.strip().split()] for label in decoded_labels]

    # 4. Calculate BLEU with smoothing
    # Smoothing is important early in training when 4-gram matches are rare
    smoothie = SmoothingFunction().method1
    score = corpus_bleu(formatted_labels, formatted_preds, smoothing_function=smoothie)

    return {"bleu": score * 100} # Scale to 0-100 like standard benchmarks

In [ ]:
from sklearn.model_selection import KFold
df = pd.read_csv("/kaggle/input/deep-past-initiative-machine-translation/train.csv")



train_df,val_df = train_test_split(df,test_size = 0.3)


raw_datasets = DatasetDict({
"train": Dataset.from_pandas(train_df),
"validation": Dataset.from_pandas(val_df)
    
})
    
    
print("Sample Transliteration:", train_df.iloc[0]['transliteration'],'\n')
print("Sample Translation:", train_df.iloc[0]['translation'])

tokenized_datasets = raw_datasets.map(
preprocess_function, 
batched=True,
remove_columns=raw_datasets["train"].column_names
)
print("Tokenization Complete.")

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics
    )

print("Starting Memory-Optimized Training...")
trainer.train()
    # SAVE
final_path = "./flant5_small_final_optimized"
trainer.save_model(final_path)
tokenizer.save_pretrained(final_path)
print(f"Saved model to {final_path}")

Sample Transliteration: šu-ma um-me-a-an ša-lim-a-šur a-na AN.NA ù x x x na ší TÚG ša en-na-nim x zu ú en-um-a-šur a-na en-na-nim i-tù-ru ma-lá AN.NA ù (TÚG)ṣú-ba-tí ša qá-tim i-na ba-áb DINGIR-lim i-za-ku-ú iš-tí um-mì-a-an ša-lim-a-šur en-na-nam DUMU am-ri-a en-um-a-šur ú-ba-áb IGI tù-ur-a-a IGI lá-qé-ep 

Sample Translation: If Šalim-Ašsur's investors raise claim against Ennānum concerning tin ... textiles belonging to Ennānum concerning tin ... textiles belonging to Ennānum ... and Ennam-Aššur, then Ennam-Aššur will release Ennānum son of Amriya with Šalim-Aššur's investors for as much tin and ordinary textiles as they clear at the gate of the God. Witnessed by Turaya, by Lā-qēp.


Map:   0%|          | 0/1092 [00:00<?, ? examples/s]

Map:   0%|          | 0/469 [00:00<?, ? examples/s]

Tokenization Complete.
Starting Memory-Optimized Training...


/tmp/ipykernel_55/2796278204.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu
1,No log,0.297040,37.510941
2,0.189200,0.290125,43.350021
3,0.170600,0.299471,43.867495


/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


In [ ]:
final_path = "./flanT5_small_CV_optimized"
trainer.save_model(final_path)
tokenizer.save_pretrained(final_path)
print(f"Saved model to {final_path}")

In [ ]:
import os
import re
import unicodedata
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# --- CONFIGURATION ---
TEST_PATH = "/kaggle/input/deep-past-initiative-machine-translation/test.csv"
MODEL_PATH = "/kaggle/working/flant5_small_final_optimized" 

BATCH_SIZE = 16   # Safe inference batch size
MAX_LENGTH = 384  # Must match training length
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --- CLEANING FUNCTION (Must match training!) ---
def clean_text(text):
    if pd.isna(text): return ""
    text = str(text)
    text = unicodedata.normalize('NFC', text)
    text = text.replace("…", "...")
    text = re.sub(r'\.\s*\.\s*\.', '...', text)
    text = text.replace('“', '"').replace('”', '"')
    text = re.sub(r'\s+', ' ', text).strip()
    return text

print(f"Loading model from {MODEL_PATH}...")

# Load Model Offline
try:
    tokenizer = AutoTokenizer.from_pretrained('/kaggle/working/flant5_small_final_optimized', local_files_only=True)
    model = AutoModelForSeq2SeqLM.from_pretrained('/kaggle/working/flant5_small_final_optimized', local_files_only=True)
    model = model.to(DEVICE)
    model.eval()
    print("✅ Model loaded successfully!")
except Exception as e:
    print(f"❌ Error loading model: {e}")

In [ ]:
class InferenceDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.texts = [clean_text(t) for t in df['transliteration']]
        # Add prefix
        self.texts = ["translate Akkadian to English: " + t for t in self.texts]
        self.tokenizer = tokenizer
        self.ids = df['id'].tolist()
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer(
            text, 
            padding="max_length", 
            truncation=True, 
            max_length=MAX_LENGTH, 
            return_tensors="pt"
        )
        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "id": self.ids[idx]
        }

# Load Test Data
test_df = pd.read_csv(TEST_PATH)
test_dataset = InferenceDataset(test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

all_predictions = []
all_ids = []

print(f"Starting Inference on {len(test_df)} rows...")

with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        
        # Beam Search for best translation quality
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=MAX_LENGTH,
            num_beams=4,
            early_stopping=True
        )
        
        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        all_predictions.extend([d.strip() for d in decoded])
        all_ids.extend(batch["id"].tolist())

print("Inference Complete.")

In [ ]:
# Create DataFrame
submission = pd.DataFrame({
    "id": all_ids,
    "translation": all_predictions
})

# Final Sanity Check: Fill empty predictions if any exist
submission["translation"] = submission["translation"].apply(lambda x: x if len(x) > 0 else "...")

# Save
submission.to_csv("submission.csv", index=False)
print("submission.csv saved.")
print(submission.head())